In [ ]:
import wget
url = 'http://www.image-net.org/archive/wordnet.is_a.txt'
wget.download(url, out = 'parent_child.txt')

In [12]:
url2 = 'http://image-net.org/archive/words.txt'
wget.download(url2, out = 'map_wordnet.txt')

'map_wordnet.txt'

In [1]:
num_lines = sum(1 for line in open('parent_child.txt'))
num_lines

75850

In [2]:
num_lines2 = sum(1 for line in open('map_wordnet.txt'))
num_lines2

82115

In [3]:
i = 0
with open('map_wordnet.txt') as f:
    while i < 5:
        line = f.readline()
        print line
        i += 1

n00001740	entity

n00001930	physical entity

n00002137	abstraction, abstract entity

n00002452	thing

n00002684	object, physical object



In [4]:
line[10:-1]

'object, physical object'

In [5]:
id_labels = {}
with open('map_wordnet.txt') as f:
    for line in f:
        id_labels[line[:9]] = line[10:-1].split(', ')

In [6]:
i = 0
with open('parent_child.txt') as f:
    while i < 5:
        line = f.readline()
        print line.split()
        i += 1

['n02118333', 'n02119789']
['n02471300', 'n02478875']
['n02471762', 'n02473983']
['n02100399', 'n02100735']
['n02374149', 'n02390258']


In [7]:
# class Node(object):
#     def __init__(self, data):
#         self.data = data
#         self.children = []
#         self.parent = []

#     def add_child(self, obj):
#         self.children.append(obj)

In [8]:
dic_child = {}
dic_parent = {}

In [9]:
def add_child(parent, child):
    temp_child = dic_child.get(parent, [])
    temp_child.append(child)
    dic_child[parent] = temp_child
    temp_parent = dic_parent.get(child, [])
    temp_parent.append(parent)
    dic_parent[child] = temp_parent

In [10]:
with open('parent_child.txt') as f:
    for line in f:
        parent = line.split()[0]
        child = line.split()[1]
        add_child(parent, child)

In [11]:
len(dic_parent.keys())

74389

In [12]:
len(dic_child)

16693

In [13]:
for i, (k, v) in enumerate(dic_child.items()):
    if i > 5: break
    print k, v

n09540430 ['n09541125', 'n09546604', 'n09540939', 'n09546280', 'n09541661', 'n09541809', 'n09546772', 'n09546453', 'n09540739']
n03558404 ['n04273972', 'n03523987', 'n03336282', 'n04098710']
n06863751 ['n06864308', 'n06864192', 'n06864407']
n13035241 ['n13035707', 'n13036804']
n07786686 ['n07787429', 'n07786856']
n05048301 ['n05048833', 'n05048690', 'n05048948', 'n05049137']


In [14]:
for i, (k, v) in enumerate(dic_parent.items()):
    if len(v) > 2:
        print i, id_labels[k], k
        for x in v:
            print id_labels[x]
        print ""

3430 ['atropine'] n02754756
['antidote', 'counterpoison']
['alkaloid']
['poison', 'toxicant', 'poisonous substance']
['mydriatic', 'mydriatic drug']
['antispasmodic', 'spasmolytic', 'antispasmodic agent']

4575 ['five-spot', 'five'] n03353783
['spot']
['die', 'dice']
['domino']

5188 ['sleeper'] n10610850
['saboteur', 'wrecker', 'diversionist']
['terrorist']
['spy', 'undercover agent']

7573 ['Christmas', 'Christmas Day', 'Xmas', 'Dec 25'] n15196186
['legal holiday', 'national holiday', 'public holiday']
['quarter day']
['holy day of obligation']
['feast day', 'fete day']

10458 ['umbilical cord', 'umbilical'] n05520479
['funiculus']
['duct', 'epithelial duct', 'canal', 'channel']
['fetal membrane']

13586 ['multiple sclerosis', 'MS', 'disseminated sclerosis', 'disseminated multiple sclerosis'] n14094068
['autoimmune disease', 'autoimmune disorder']
['degenerative disorder']
['sclerosis', 'induration']

14147 ['length'] n05129201
['dimension']
['physical property']
['fundamental quanti

In [15]:
url3 = 'https://raw.githubusercontent.com/tensorflow/models/master/research/inception/inception/data/imagenet_lsvrc_2015_synsets.txt'
wget.download(url3, out = '1k_synsets.txt')

NameError: name 'wget' is not defined

In [16]:
synsets_1k = []
with open('1k_synsets.txt') as f:
    for line in f:
        synsets_1k.append(line[:9])

In [17]:
def find_ancestor(node, level):
    if level == 0:
        return set([node])
    else:
        ancestors = set([])
        if node not in dic_parent:
            return set([])
        parents = dic_parent[node]
        for parent in parents:
            ancestors = ancestors.union(find_ancestor(parent, level-1))
    return ancestors

def find_descendant(node, level):
    if level == 0:
        return set([node])
    else:
        descendants = set([])
        if node not in dic_child:
            return set([])
        children = dic_child[node]
        for child in children:
            descendants = descendants.union(find_descendant(child, level-1))
    return descendants
        

In [18]:
def hop(node, up, down):
    ancestors = find_ancestor(node, up)
    set_hop = set([])
    for ancestor in ancestors:
        for level in range(down+1):
            set_hop = set_hop.union(find_descendant(ancestor, level))
    if node in set_hop:
        set_hop.remove(node)
    return set_hop

In [19]:
def is_leaf(node):
    return node not in dic_child

In [20]:
hop2 = set([])
for synset in synsets_1k:
    hop2 = hop2.union(hop(synset, 2, 2))
    hop2_leaf = set((filter(lambda x: is_leaf(x), hop2)))     

In [21]:
sum(is_leaf(x) for x in synsets_1k)

650

In [22]:
dic_child['n02342885']

['n02343320', 'n02343058']

# New Hop

In [23]:
def hop_dist(node, dist):
    set_hop = set([])
    for d in range(dist+1):
        for up in range(1,d+1):
            down = d - up
            ancestors = find_ancestor(node, up)
            for ancestor in ancestors:
                set_hop = set_hop.union(find_descendant(ancestor, down))
    return set_hop

In [24]:
hop_dist('n02342885', 1)

{'n02329401'}

In [25]:
hop2 = set([])
for synset in synsets_1k:
    hop2 = hop2.union(hop_dist(synset, 2))
hop2 -= set(synsets_1k)

In [26]:
len(hop2)

5242

In [27]:
sum(is_leaf(n) for n in hop2)

3439

In [28]:
not_leaf2 = 0
for synset in synsets_1k:
    if not is_leaf(synset):
        if find_descendant(synset, 3) != set([]):
            not_leaf2 += 1
            print id_labels[synset]
            print [id_labels[x] for x in find_descendant(synset, 2)]

['vulture']
[['buzzard', 'turkey buzzard', 'turkey vulture', 'Cathartes aura'], ['black vulture', 'carrion crow', 'Coragyps atratus'], ['Egyptian vulture', "Pharaoh's chicken", 'Neophron percnopterus'], ['condor'], ['king vulture', 'Sarcorhamphus papa'], ['black vulture', 'Aegypius monachus'], ['griffon vulture', 'griffon', 'Gyps fulvus'], ['bearded vulture', 'lammergeier', 'lammergeyer', 'Gypaetus barbatus']]
['fly']
[['horsefly', 'cleg', 'clegg', 'horse fly'], ['botfly'], ['greenbottle', 'greenbottle fly'], ['warble fly'], ['bluebottle', 'Calliphora vicina']]
['grasshopper', 'hopper']
[['locust'], ['sand cricket', 'Jerusalem cricket', 'Stenopelmatus fuscus'], ['katydid']]
['knot']
[['Windsor knot'], ['Matthew Walker', 'Matthew Walker knot'], ['timber hitch'], ['bowline', 'bowline knot'], ["cat's-paw"], ['Blackwall hitch'], ['sheet bend', 'becket bend', "weaver's knot", "weaver's hitch"], ['rolling hitch'], ['reef knot', 'flat knot']]
['missile']
[['Exocet'], ['space probe'], ['buzz b

In [29]:
not_leaf2

14

In [30]:
sum(is_leaf(x) for x in synsets_1k)

650

In [31]:
num_leaf_full = 0
set_dict_keys = set([])
with open('map_wordnet.txt') as f:
    for line in f:
        synset = line[:9]
        set_dict_keys.add(synset)
        num_leaf_full += is_leaf(synset)

In [32]:
num_leaf_full

65422

In [33]:
set_all_synset = set(dic_child.keys()).union(set(dic_parent.keys()))

In [34]:
len(set_all_synset)

74401

In [35]:
sum(is_leaf(x) for x in set_all_synset)

57708

In [36]:
set_all_synset.issubset(set_dict_keys)

True

In [37]:
def hop_dist_simp(node, dist):
    set_hop = set([])
    for d in range(dist+1):
        for up in range(0,d+1):
            down = d - up
            ancestors = find_ancestor(node, up)
            for ancestor in ancestors:
                set_hop = set_hop.union(find_descendant(ancestor, down))
    return set((filter(lambda x: is_leaf(x), set_hop)))  

In [38]:
hop_dist_simp('n02342885', 1)

{'n02343058', 'n02343320'}

In [39]:
hop2_simp = set([])
for synset in synsets_1k:
    hop2_simp = hop2_simp.union(hop_dist_simp(synset, 2))
hop2_simp -= set(synsets_1k)

In [40]:
len(hop2_simp)

4620

In [41]:
set((filter(lambda x: is_leaf(x), hop2))).issubset(hop2_simp)

True

In [42]:
hop_down_2 = set([])
for synset in synsets_1k:
    hop_down_2 = hop_down_2.union(find_descendant(synset, 2))
len(set((filter(lambda x: is_leaf(x), hop_down_2))))

238

In [43]:
def is_one_word(synset):
    labels = id_labels[synset]
    res = 1
    for label in labels:
        if ' ' not in label:
            res *= 0
        else:
            res *= 1
    return not res
    
#     return not reduce((lambda x, y: (' ' in x) * (' ' in y)), labels, '')

In [49]:
get_one_word('n02343058')

('hamster', False)

In [50]:
is_one_word('n02421612')

True

In [51]:
id_labels['n02421612']

['Madoqua', 'genus Madoqua']

In [315]:
len(set((filter(lambda x: is_one_word(x), hop2_simp))))

2744

In [47]:
def get_recent_oneword_ancestor(synset):
    level = 0
    while level >= 0:
        for ancestor in find_ancestor(synset, level):
            if is_one_word(ancestor):
                return ancestor
            level += 1
    
#     level = 0
#     if is_one_word(synset):
#         return (synset, level)
#     else:
#         level += 1
#         parents = dic_parent[synset]
#         return min([get_recent_oneword_ancestor(parent) for parent in parents], key = lambda x: x[1])
    

In [48]:
def get_one_word(synset):
    curr = get_recent_oneword_ancestor(synset)
    labels = id_labels[curr]
    for label in labels:
        if ' ' not in label:
            return label, is_one_word(synset)

In [325]:
get_recent_oneword_ancestor('n02343058')

'n02342885'

In [327]:
set((filter(lambda x: is_one_word(x), hop2_simp)))

{'n03024882',
 'n04073425',
 'n04354589',
 'n03906894',
 'n01541760',
 'n02824058',
 'n07593471',
 'n09407194',
 'n04123123',
 'n03260733',
 'n02486908',
 'n04439505',
 'n09266604',
 'n03246454',
 'n03710079',
 'n02888270',
 'n04521863',
 'n03356559',
 'n01859325',
 'n03316105',
 'n07872593',
 'n04237287',
 'n07746038',
 'n02513355',
 'n04230487',
 'n03655295',
 'n04545748',
 'n09478962',
 'n07835823',
 'n02129530',
 'n04520480',
 'n02233943',
 'n01639187',
 'n02775359',
 'n07801091',
 'n03467380',
 'n03254862',
 'n04598318',
 'n03524287',
 'n07745357',
 'n03999064',
 'n03059934',
 'n03549350',
 'n04176747',
 'n03164722',
 'n04030274',
 'n04249415',
 'n07919787',
 'n03531691',
 'n04347119',
 'n04409279',
 'n03548402',
 'n04052235',
 'n03364599',
 'n03264906',
 'n11896141',
 'n01824344',
 'n03776167',
 'n03963294',
 'n03115762',
 'n01834540',
 'n04374735',
 'n04300643',
 'n07879450',
 'n03548533',
 'n02936402',
 'n01932936',
 'n02871631',
 'n02422391',
 'n03797182',
 'n02492948',
 'n031

In [340]:
output = 'hop2.txt'
f = open(output, "w+")
for synset in set((filter(lambda x: is_one_word(x), hop2_simp))):
    print >> f, synset
f.close()

In [336]:
for i in set((filter(lambda x: is_one_word(x), hop2_simp))):
    if len(i) != 9:
        print i